In [25]:
import os #os module for interacting with locally saved files.
import re #regex is needed for separating schedule types.
import pandas as pd #pandas dataframes are use to represent the tabular data of the rotas.
from datetime import datetime #datetime is used for creating datetime objects from data within the rota that have been represented as strings.
import calendar #calendar is used to identify days in specific calendar months to avoid mistakes in calculating date maths.

#declaring variables
spreadsheets = []
rotas = []
holidays = []

#INPUT NEEDED HERE
directory = 'Example Rotas' #PATH TO THE FOLDER CONTAINING CSV FILES: [ROTAS, HOLIDAY SCHEDULE]
spreadsheets = os.listdir(directory) #save a list of all the files in the chosen folder
weekend_threshold = 2 #this is the independent variable to choose! = how many weekends is someone allowed to work in a row

# ALL KEY FUNCTIONS
def format_rota(df):
    '''
    Requires 1 passed argument: a dataframe representation of the csv file containing a rota in the following format: https://github.com/th3ssalus/RobustRotaChecker/raw/main/Rota%20Template.xlsx.
        Format is a standard Starbucks rota, containing the first name of employee and Starbucks partner number.
    Functionality: Standardises column names and applies contextual labels to improve navigating the rota.
    '''
    df.drop([0,1], inplace=True)
    df.dropna(axis=1,how='all',inplace=True)
    df.rename(columns={'Unnamed: 2':'Partner','Unnamed: 3':'Role', 'Unnamed: 4':'Contract Hours','Unnamed: 6':'Sunday Start', 'Unnamed: 7':'Sunday Finish','Unnamed: 8':'Sunday Hours','Unnamed: 9':'Monday Start','Unnamed: 10':'Monday Finish','Unnamed: 11':'Monday Hours','Unnamed: 12':'Tuesday Start','Unnamed: 13':'Tuesday Finish','Unnamed: 14':'Tuesday Hours','Unnamed: 15':'Wednesday Start','Unnamed: 16':'Wednesday Finish','Unnamed: 17':'Wednesday Hours','Unnamed: 18':'Thursday Start','Unnamed: 19':'Thursday Finish','Unnamed: 20':'Thursday Hours','Unnamed: 21':'Friday Start','Unnamed: 22':'Friday Finish','Unnamed: 23':'Friday Hours','Unnamed: 24':'Saturday Start','Unnamed: 25':'Saturday Finish','Unnamed: 26':'Saturday Hours','Unnamed: 27':'Weekly Total Hours'},inplace = True)
    df.rename(columns={'Sunday Hours':df.iloc[0]['Sunday Hours'],'Monday Hours':df.iloc[0]['Monday Hours'],'Tuesday Hours':df.iloc[0]['Tuesday Hours'],'Wednesday Hours':df.iloc[0]['Wednesday Hours'],'Thursday Hours':df.iloc[0]['Thursday Hours'],'Friday Hours':df.iloc[0]['Friday Hours'],'Saturday Hours':df.iloc[0]['Saturday Hours']},inplace = True)
    df.drop([2,3],inplace = True)
    df.dropna(subset = 'Partner',inplace=True)
    return df

def weekend_format(df):
    '''
    Requires 1 passed argument: a dataframe representation of the csv file containing a rota in the following format: https://github.com/th3ssalus/RobustRotaChecker/raw/main/Rota%20Template.xlsx.
        Format is a standard Starbucks rota, containing the first name of employee and Starbucks partner number.
    Functionality: Splits the rota into week and weekend and returns a tuple: (days of the week as strings, an independent dataframe containing weekend data from the rota)
    '''
    header = list(df)
    week = header[5]
    weekends_df = df[['Partner','Sunday Start','Saturday Start']].copy()
    return week, weekends_df

def weekends_worked(rotas):
    '''
    Requires 1 passed argument: a list of rotas (i.e. the "spreadsheets" variable).
    Functionality: Iterates through the rotas in the folder, separates weekend data, identifies employees who have are over the threshold for weekends worked in a row.
    '''
    overworked_partners = []
    
    x = []
    for rota in set(rotas):
        df = pd.read_excel(f'{directory}/{rota}')
        x.append(weekend_format(format_rota(df)))

    weekend_workers=[]
    z=[]
    for item in x:
        for index,row in item[1].iterrows():
            if pd.isnull(row['Sunday Start']) or pd.isnull(row['Saturday Start']):
                None
            else:
                weekend_workers.append([row['Partner'],item[0]])        
    weekend_workers=sorted(weekend_workers)

    previous_entry = []
    repeaters=[]
    for entry in weekend_workers:
        if previous_entry == []:
            previous_entry = entry
            continue
        elif entry[0] == previous_entry[0]:
            repeaters.append([entry[0],previous_entry[1],entry[1]])
        previous_entry=entry

    #comparing weekend shifts between rotas
    for repeater in repeaters:
        if len(repeater) == weekend_threshold+1:
            overworked_partners.append(repeater[0])
    return set(overworked_partners)


#organise spreadsheets into type
for spreadsheet in spreadsheets:
    if 'holiday' in spreadsheet.lower():
        holidays.append(spreadsheet)
        continue
    elif re.search("^[0-3][0-9]",spreadsheet) != None:
        rotas.append(spreadsheet)
#identify who should not be working another weekend
# weekends_worked(rotas)

In [27]:
#HOLIDAY CHECKER
def holiday_format(df):
    '''
    Requires 1 passed argument: a dataframe representation of the csv file containing a rota in the following format: https://github.com/th3ssalus/RobustRotaChecker/raw/main/270823-230923%20holiday.xlsx
    Functionality: Standardises column names and applies contextual labels to improve navigating the rota.
    '''
    df.columns = df.iloc[0]
    df = df[1:]
    return df

def holidayfilecheck(holidays,rotas):
    '''
    Requires 2 passed arguments: 
        1. the list of holiday spreadsheets,
        2. the list of rota spreadsheets.
    Functionality: Verifies that the date range for the holiday spreadsheet is aligned with the rotas present in the folder and alerts user if this isn't true.
    '''
    rotas.sort(key=lambda date: datetime.strptime(date, "%d%m%y.xlsx")) #organise rota array in date order
    for holiday in holidays:
        if rotas[0].split('.')[0] in holiday: #check the first rota's date matches up with the earliest date in the holiday
            startdate=holiday[0:6]
            return holiday,startdate
    
        else:
            print('You need to check the date of the holiday spreadsheet and the rotas are the same.')
            return False

def holidayadvocate(holidays,rotas):
    '''
    Requires 2 passed arguments:
        1. a list of the holiday schedules represented as dataframes,
        2. a list of the rota schedules represented as dataframes.
    Functionality: Returns a dataframe of the holiday schedule and a timestamp of the startdate of the holiday date range.
    '''
    # grab all of the rotas and put them into dataframes
    # Collect all the dataframes in a list.
    x=[]
    for rota in set(rotas):
        df = pd.read_excel(f'{directory}/{rota}')
        x.append(format_rota(df))
    
    #Check holiday date range and rota date range are in alignment by date.
    # And return the path for the holiday spreadsheet and it's startdate.
    (filename,startdate) = holidayfilecheck(holidays,rotas)
    # convert the holiday scheudle into a dataframe.
    holiday = pd.read_excel(f'{directory}/{filename}')
    #holiday = holiday_format(holiday)

    #return the holiday dataframe and it's startdate
    return holiday,startdate
    #print(holiday.notnull())

def startdatemath(startdate):
    '''
    Requires 1 passed argument: the startdate of a schedule.
    Functionality: Calculates how many days are in a given month to enable accurate date maths and returns it as a tuple: (month as an integer, days in that month)
    '''
    #how many days in that given month?
    return (int(startdate[3:4]),calendar.monthrange(int(f'20{startdate[4:6]}'), int(startdate[3:4]))[1])

In [29]:
(holiday,startdate) = holidayadvocate(holidays,rotas)
#holiday.rename(columns={'Unnamed: 0':'Partner'},inplace=True)
#grouped = holiday.groupby(df.Partner)
#df1 = grouped.get_group("Oasis 124619")
startdatemath(startdate)

(8, 31)

In [31]:
holidays

['270823-230923 holiday.xlsx']

In [33]:
dates = ['1st','2nd','3rd','4th','5th','6th','7th','8th','9th','10th','11th','12th','13th','14th','15th','16th','17th','18th','19th','20th','21st','22nd','23rd','24th','25th','26th','27th','28th','29th','30th','31st']
holidates = ['Partner','Sunday','Monday','Tuesday','Wednesday','Thursday','Friday','Saturday']
date_map = {}
h=0
checker = {}
days= []
(month,daysinmonth) = startdatemath(startdate)

for index,row in holiday.iterrows():
    #if we see a 1 we change the month    
    #this changes the dates as we move down each week
    if row[1] in dates:
        for item in row:
            if item == '1st':
                month = month+1
            date_map[holidates[h]] = f'{item} of {calendar.month_name[month]}'
            h+=1
        h=0
    elif type(row[0]) != str: #this stops non strings from being included in the final list, effectively getting rid of null values
        continue
    #this adds the days of holiday found to a final output
    else:
        #cut off the head and remove null values
        body = row[1:8].dropna()
        if len(body) > 0:
            #print(f'{row[0]}: {body}')  
            for day in body.index.tolist():
                days.append(f'{day} the {date_map[day]}')
                
            checker[row[0]]=days
            days=[]
            
checker

C:\Users\harry\AppData\Local\Temp\ipykernel_13300\2351463210.py:12: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if row[1] in dates:
C:\Users\harry\AppData\Local\Temp\ipykernel_13300\2351463210.py:19: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  elif type(row[0]) != str: #this stops non strings from being included in the final list, effectively getting rid of null values
C:\Users\harry\AppData\Local\Temp\ipykernel_13300\2351463210.py:30: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior).

{'James 133231': ['Sunday the 27th of August'],
 'Terri 122343': ['Sunday the 3rd of September',
  'Monday the 4th of September',
  'Tuesday the 5th of September',
  'Wednesday the 6th of September',
  'Thursday the 7th of September',
  'Friday the 8th of September',
  'Saturday the 9th of September'],
 'John 152444': ['Sunday the 3rd of September'],
 'Oasis 124619': ['Sunday the 17th of September'],
 'Louise 163383': ['Tuesday the 19th of September']}

In [35]:
# Helper function to strip the suffix from the day
def strip_day_suffix(day_str):
    return int(re.sub(r'(st|nd|rd|th)', '', day_str))

# Function to parse and compare dates
def parse_date(day, month, year):
    return pd.to_datetime(f'{day}-{month}-{year}', format='%d-%B-%Y')

# Function to find conflicts between holidays and rotas
def find_conflicts(checker, rotas):
    conflicts = []
    year = startdate[-2:]  # Extract year from startdate

    for rota in set(rotas):
        df = pd.read_excel(f'{directory}/{rota}')
        df = format_rota(df)

        rota_week = pd.to_datetime(rota.split('.')[0], format='%d%m%y')
        rota_start_date = rota_week - pd.to_timedelta(rota_week.weekday(), unit='D')
        rota_dates = pd.date_range(rota_start_date, periods=7).strftime('%Y-%m-%d').tolist()

        for person, holidays in checker.items():
            person_rota = df[df['Partner'] == person]

            if not person_rota.empty:
                for holiday in holidays:
                    day_of_week, day_and_month = holiday.split(' the ')
                    day = strip_day_suffix(day_and_month.split()[0])
                    month = day_and_month.split()[2]
                    holiday_date = parse_date(day, month, f'20{year}')

                    # Check if holiday_date matches any dates in rota
                    if holiday_date.strftime('%Y-%m-%d') in rota_dates:
                        work_start = person_rota[f'{day_of_week} Start'].values[0]
                        if not pd.isnull(work_start):
                            conflicts.append((person, holiday_date.strftime('%Y-%m-%d'), rota))

    
    return conflicts

# Run the conflict check
conflicts = find_conflicts(checker, rotas)

# Output results of the holiday check
if conflicts:
    print('The following holiday conflicts were found:')
    for conflict in conflicts:
        print(f" *Conflict: {conflict[0]} is scheduled to work on {conflict[1]} (Rota: {conflict[2]})")
else:
    print("No holiday conflicts found.")

# Output the results of the weekend check
print('\n')
print(f'The following people should not be scheduled to work the weekend in the next rota after {rotas[-1][0:2]}-{rotas[-1][2:4]}-{rotas[-1][4:6]}:')
for person in weekends_worked(rotas):
    print(f' *{person}')



The following holiday conflicts were found:
 *Conflict: Oasis 124619 is scheduled to work on 2023-09-17 (Rota: 170923.xlsx)
 *Conflict: Terri 122343 is scheduled to work on 2023-09-06 (Rota: 100923.xlsx)
 *Conflict: Terri 122343 is scheduled to work on 2023-09-07 (Rota: 100923.xlsx)
 *Conflict: Terri 122343 is scheduled to work on 2023-09-08 (Rota: 100923.xlsx)


The following people should not be scheduled to work the weekend in the next rota after 17-09-23:
 *Tom 153229
 *Harry 163437
 *Emily 146217
 *Louise 163383
 *Oasis 124619
